
# 📒 Kraken Training on Colab — **ALTO** Makefile-Style (msia compatible)

This notebook is tailored for **ALTO exports** (eScriptorium-style). It lets you:
- Upload one or more **ZIPs** from Finder (auto-extracted)
- (Optional) add Google Drive folders
- Train **segmentation** and **recognition** with **custom learning rates**
- Evaluate recognition (CER/WER)
- Package outputs and (optionally) upload to **msia.escriptorium.fr**

> In Colab: **Runtime → Change runtime type → GPU** before running.


In [ ]:

# 0) GPU check
!nvidia-smi || true


## 1) Install Kraken + deps (Python 3.12 safe)

In [ ]:

import sys, subprocess
def pip_install(*pkgs):
    print("pip install", " ".join(pkgs))
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", *pkgs])
KR_PIN = "kraken==5.3.0"
pip_install(KR_PIN, "torch>=2.1,<3", "cairocffi", "opencv-python", "lxml", "h5py")


In [ ]:

# Verify installs
from importlib.metadata import version, PackageNotFoundError
import torch, shutil, sys
def pkg_ver(name):
    try: return version(name)
    except PackageNotFoundError: return "not installed"
print("python:", sys.version.split()[0])
print("kraken:", pkg_ver("kraken"))
print("torch:", torch.__version__)
print("cuda available:", torch.cuda.is_available())
print("ketos on PATH:", shutil.which("ketos"))
!ketos --version || true


## 2) Config — ALTO fixed, roots, and split

In [ ]:

# ======= CONFIG (ALTO) =======
FORMAT = "alto"  # hard-coded for ALTO exports

# Optional: add Drive roots if you also keep ALTO data on Drive
DRIVE_ROOTS = [
    # "/content/drive/MyDrive/my_alto_export_folder"
]

# Validation split fraction
VAL_FRACTION = 0.10
# =============================


### (Optional) Mount Google Drive if you use DRIVE_ROOTS

In [ ]:

from google.colab import drive
drive.mount('/content/drive')


## 3) Upload ALTO ZIP(s) from Finder — auto-extract & discover roots

In [ ]:

import os, io, zipfile, re
from google.colab import files

LOCAL_BASE = "/content/data_alto"
os.makedirs(LOCAL_BASE, exist_ok=True)

print("Upload one or more ALTO export ZIPs from Finder. They will be extracted into", LOCAL_BASE)
uploaded = files.upload()

EXTRACTED_ROOTS = []
for name, data in uploaded.items():
    save_path = os.path.join(LOCAL_BASE, name)
    with open(save_path, "wb") as f:
        f.write(data)
    if name.lower().endswith(".zip"):
        base = re.sub(r"\s*\(\d+\)\s*$", "", os.path.splitext(name)[0])
        target_dir = os.path.join(LOCAL_BASE, base)
        os.makedirs(target_dir, exist_ok=True)
        with zipfile.ZipFile(io.BytesIO(data)) as zf:
            zf.extractall(target_dir)
        os.remove(save_path)
        EXTRACTED_ROOTS.append(target_dir)
    else:
        EXTRACTED_ROOTS.append(LOCAL_BASE)

print("Extracted roots:", EXTRACTED_ROOTS if EXTRACTED_ROOTS else ["(none; you can still use DRIVE_ROOTS)"])


## 4) Build merged ALTO XML list (case-insensitive) + sanity checks

In [ ]:

import os, glob, xml.etree.ElementTree as ET

ROOTS = [p for p in (EXTRACTED_ROOTS + DRIVE_ROOTS) if p and os.path.exists(p)]
assert ROOTS, "No existing roots found. Upload a ZIP or set DRIVE_ROOTS."

def find_alto_xmls(root):
    xmls = glob.glob(os.path.join(root, "**", "*.xml"), recursive=True) +            glob.glob(os.path.join(root, "**", "*.XML"), recursive=True)
    out = []
    for xp in xmls:
        try:
            r = ET.parse(xp).getroot()
            if isinstance(r.tag, str) and "alto" in r.tag.lower():
                out.append(xp)
        except ET.ParseError:
            pass
    return sorted(out)

xmls = []
for root in ROOTS:
    found = find_alto_xmls(root)
    print(f"[ALTO] {len(found):>5} XMLs in {root}")
    xmls.extend(found)

print("TOTAL ALTO XML files:", len(xmls))
assert len(xmls) >= 3, "Need at least a few ALTO XML pages."

# Heuristic image presence check (by filename stem match)
import collections
stems = collections.Counter(os.path.splitext(os.path.basename(p))[0] for p in xmls)
img_exts = {".jpg",".jpeg",".tif",".tiff",".png",".jp2",".bmp"}
img_candidates = []
for root in ROOTS:
    for ext in img_exts:
        img_candidates += glob.glob(os.path.join(root, "**", f"*{ext}"), recursive=True)
img_stems = set(os.path.splitext(os.path.basename(p))[0] for p in img_candidates)
missing = [s for s in stems if s not in img_stems]
print("Heuristic: ALTO page stems without matching image file:", len(missing))
print("First few missing stems:", missing[:10])


## 5) Create explicit train/val lists

In [ ]:

import random, pathlib
random.seed(42)

random.shuffle(xmls)
n_val = max(1, int(len(xmls)*VAL_FRACTION))
val_xmls = xmls[:n_val]
train_xmls = xmls[n_val:]

pathlib.Path("/content/lists").mkdir(parents=True, exist_ok=True)
open("/content/lists/train.txt","w").write("\n".join(train_xmls))
open("/content/lists/val.txt","w").write("\n".join(val_xmls))

len(train_xmls), len(val_xmls)


## 6) Hyperparameters (+ auto device)

In [ ]:

# ======= EDIT ME (Hyperparameters) =======
import torch
DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"
print("Using device:", DEVICE)

# Recognition
REC_EPOCHS = 30
REC_BATCH  = 8
REC_OPTIM  = "Adam"   # "Adam" or "SGD"
REC_LR     = 1e-4     # If your Kraken errors on --lr, use --lrate in the command cell
REC_WD     = 1e-5     # weight decay

# Segmentation
SEG_EPOCHS = 20
SEG_BATCH  = 2
SEG_OPTIM  = "Adam"
SEG_LR     = 5e-4
# =========================================


## 7) Target: `train_seg` — Segmentation training

In [ ]:

# Create output dir
!mkdir -p /content/models


In [ ]:

# ===== train_seg =====
# To adjust learning rate, modify --lr (or --lrate if your Kraken is older).
# To change epochs, modify --epochs.
# To change batch-size, modify --batch-size.
# To continue training from a prior segmentation model, add:
#    --load-model /path/to/seg_base.mlmodel

import shlex, os

seg_cmd = f'''
ketos segtrain -f {shlex.quote(FORMAT)} @/content/lists/train.txt \
  -o /content/models/seg \
  --device {shlex.quote(DEVICE)} \
  --epochs {SEG_EPOCHS} \
  --batch-size {SEG_BATCH} \
  --optimizer {shlex.quote(SEG_OPTIM)} --lr {SEG_LR} \
  --validation @/content/lists/val.txt
'''
print(seg_cmd)
os.system(seg_cmd)


## 8) Target: `train_recog` — Recognition training

In [ ]:

# ===== train_recog =====
# To adjust learning rate, modify --lr (or --lrate if your Kraken is older).
# To change epochs, modify --epochs.
# To change batch-size, modify --batch-size.
# To continue training from a prior recognizer, add:
#    --load-model /path/to/rec_base.mlmodel

import shlex, os

rec_cmd = f'''
ketos train -f {shlex.quote(FORMAT)} @/content/lists/train.txt \
  -o /content/models/rec \
  --device {shlex.quote(DEVICE)} \
  --epochs {REC_EPOCHS} \
  --batch-size {REC_BATCH} \
  --optimizer {shlex.quote(REC_OPTIM)} --lr {REC_LR} --weight-decay {REC_WD} \
  --validation @/content/lists/val.txt
'''
print(rec_cmd)
os.system(rec_cmd)


## 9) Target: `eval` — Evaluate recognition (CER/WER)

In [ ]:

# ===== eval =====
# Evaluate the best recognition model on the held-out list:

import shlex, os
eval_cmd = f'''
ketos test -f {shlex.quote(FORMAT)} /content/models/rec_best.mlmodel @/content/lists/val.txt
'''
print(eval_cmd)
os.system(eval_cmd)


## 10) Package models

In [ ]:

!cd /content/models && ls -lh && zip -r ../trained_models.zip . && cd /content


## 11) Optional: Upload to msia.escriptorium.fr via API

In [ ]:

# UI upload (My Models → Upload) is simplest.
# This cell shows how to POST a model programmatically if your instance supports it.

MSIA_URL   = "https://msia.escriptorium.fr"
API_TOKEN  = "PASTE_YOUR_TOKEN_HERE"   # keep secret; or leave blank and use UI upload
MODEL_PATH = "/content/models/rec_best.mlmodel"   # or seg_best.mlmodel
MODEL_NAME = "rec_best"

if API_TOKEN and API_TOKEN != "PASTE_YOUR_TOKEN_HERE":
    import requests
    headers = {"Authorization": f"Token {API_TOKEN}"}
    files = {"file": (MODEL_NAME + ".mlmodel", open(MODEL_PATH, "rb"), "application/octet-stream")}
    data  = {"name": MODEL_NAME}
    resp = requests.post(f"{MSIA_URL}/api/models/", headers=headers, files=files, data=data)
    print("Status:", resp.status_code)
    try:
        print(resp.json())
    except Exception:
        print(resp.text[:800])
else:
    print("Skipping API upload. Use UI upload or paste a valid API token.")
